# EMA - Stable Isotope Labeling Analysis Notebook

### Parameters
The next code block sets parameters that are used throughout the remainder of the notebook.

In [ ]:
# set metatlas generated output parameters 
project_directory = None
experiment = None
polarity = None # 'positive' or 'negative'
workflow_name = None
rt_alignment_number = None
analysis_number = None

# use None if user is the same as the creator of the original EMA output
user = None

# Pair short groups by sample type.
# First element in tuple is the unlabeled sample type, and the second is the labeled version
short_group_pairs = [(None, None),
                     (None, None),
                     (None, None)]

In [ ]:
# pylint: disable=wrong-import-position,import-error,missing-class-docstring
parameters = {k: v for k, v in globals().items() if k[0] != "_" and k not in ["In", "Out", "get_ipython", "exit", "quit", "open"]}

import logging  # noqa: E402
from pathlib import Path  # noqa: E402


class StopExecution(Exception):
    def _render_traceback_(self):
        pass


logger = logging.getLogger("metatlas.jupyter")
kernel_def = """{"argv":["shifter","--entrypoint","--image=ghcr.io/biorack/metatlas/metatlas_shifter:latest","/usr/local/bin/python","-m",
                 "ipykernel_launcher","-f","{connection_file}"],"display_name": "Metatlas Targeted","language": "python",
                 "metadata": { "debugger": true }}"""
kernel_file_name = Path.home() / ".local" / "share" / "jupyter" / "kernels" / "metatlas-targeted" / "kernel.json"
try:
    has_root_kernel = Path("/root/.local/share/jupyter/kernels/papermill/kernel.json").is_file()
except PermissionError:
    has_root_kernel = False
if not has_root_kernel and not kernel_file_name.is_file():
    kernel_file_name.parent.mkdir(parents=True, exist_ok=True)
    with kernel_file_name.open(mode="w", encoding="utf-8") as f:
        f.writelines(kernel_def)
    logger.critical('CRITICAL: Notebook kernel has been installed. Set kernel to "Metatlas Targeted" and re-run notebook.')
    raise StopExecution
try:
    from analysis_tools.stable_isotope_analysis import generate_outputs, filter_and_save_peak_heights  # noqa: E402
    from analysis_tools.noise_detection_gui import Dashboard  # noqa: E402
except ImportError as err:
    logger.critical('CRITICAL: Set notebook kernel to "Metatlas Targeted" and re-run notebook.')
    raise StopExecution from err

In [ ]:
peak_heights, compound_data, compound_keys, output_path = generate_outputs(project_directory,
                                                                           experiment,
                                                                           polarity,
                                                                           workflow_name,
                                                                           rt_alignment_number,
                                                                           analysis_number,
                                                                           short_group_pairs,
                                                                           user=user)

In [ ]:
dash = Dashboard(compound_data,
                 output_path,
                 polarity,
                 compound_idx=0,
                 plot_size=(1200, 800))

In [ ]:
filter_and_save_peak_heights(peak_heights,
                             compound_keys,
                             output_path,
                             polarity)